# NORTHWIND DATABASE

El **objetivo** de este ejercicio es generar una serie de consultas SQL que permitan extraer información relevante de la base de datos y luego exportar los resultados para construir dashboards o visualizaciones que proporcionen insights valiosos para la toma de decisiones empresariales.

La base de datos Northwind es una base de datos de ejemplo clásica que simula las operaciones de una empresa minorista. Contiene tablas relacionales que representan diferentes aspectos del negocio, como productos, clientes, pedidos y empleados. Utilizando consultas SQL, podrás explorar y analizar estos datos para comprender mejor el desempeño del negocio en diferentes áreas.

Una vez que hayas realizado las consultas SQL y obtenido los resultados, podrás utilizarlos para herramientas de visualización de datos como Tableau, Power BI, Streamlit o cualquier otra herramienta de tu elección. 

![img](./img/northwind-er-diagram.png)

Tendrás que elegir uno de los retos a resolver:

### **1. Análisis de Ventas por Región**
Analiza las ventas de productos por región. El objetivo es identificar las regiones que generan más ingresos y las tendencias de ventas a lo largo del tiempo en esas regiones. 
Pista: Utiliza gráficos de barras y líneas para mostrar la evolución de las ventas, así como mapas geográficos para visualizar la distribución de ingresos por región.

In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///data/northwind.db', echo=False)
connection = engine.connect()

In [2]:
import pandas as pd

consulta = """
SELECT datetime(o.OrderDate) as OrderDate, o.ShipRegion, od.Quantity*od.UnitPrice*(1-od.Discount) AS Total, p.ProductName
FROM Orders o
INNER JOIN [Order Details] od ON od.OrderID == o.OrderID
INNER JOIN Products p ON p.ProductID = od.ProductID
"""

data_df = pd.read_sql(consulta, con=connection)
data_df

,OrderDate,ShipRegion,Total,ProductName
0,2016-07-04 00:00:00,Western Europe,168.00,Queso Cabrales
1,2016-07-04 00:00:00,Western Europe,98.00,Singaporean Hokkien Fried Mee
2,2016-07-04 00:00:00,Western Europe,174.00,Mozzarella di Giovanni
3,2016-07-05 00:00:00,Western Europe,167.40,Tofu
4,2016-07-05 00:00:00,Western Europe,1696.00,Manjimup Dried Apples
...,...,...,...,...
609278,2023-01-10 05:40:51,British Isles,806.00,Ikura
609279,2023-01-10 05:40:51,British Isles,216.00,Spegesild
609280,2023-01-10 05:40:51,British Isles,93.69,Gumbär Gummibärchen
609281,2023-01-10 05:40:51,British Isles,1053.60,Schoggi Schokolade


In [28]:
data_df.dtypes

OrderDate       object
ShipRegion      object
Total          float64
ProductName     object
dtype: object

In [29]:
data_df["OrderDate"] = pd.to_datetime(data_df['OrderDate'])
data_df.set_index("OrderDate", inplace=True)

In [30]:
data_df.index

DatetimeIndex(['2016-07-04 00:00:00', '2016-07-04 00:00:00',
               '2016-07-04 00:00:00', '2016-07-05 00:00:00',
               '2016-07-05 00:00:00', '2016-07-08 00:00:00',
               '2016-07-08 00:00:00', '2016-07-08 00:00:00',
               '2016-07-08 00:00:00', '2016-07-08 00:00:00',
               ...
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51',
               '2023-01-10 05:40:51', '2023-01-10 05:40:51'],
              dtype='datetime64[ns]', name='OrderDate', length=609283, freq=None)

In [31]:
data_df.head()

,ShipRegion,Total,ProductName
OrderDate,,,
2016-07-04,Western Europe,168.0,Queso Cabrales
2016-07-04,Western Europe,98.0,Singaporean Hokkien Fried Mee
2016-07-04,Western Europe,174.0,Mozzarella di Giovanni
2016-07-05,Western Europe,167.4,Tofu
2016-07-05,Western Europe,1696.0,Manjimup Dried Apples


In [32]:
data_df["ShipRegion"].unique()

array(['Western Europe', 'South America', 'Central America',
       'North America', 'Northern Europe', 'Scandinavia',
       'Southern Europe', 'British Isles', 'Eastern Europe'], dtype=object)

In [33]:
reg_prod_total = data_df.groupby(["ShipRegion"]).resample("BYE").sum("Total")
reg_prod_total

Total
ShipRegion     OrderDate              
British Isles  2012-12-31   1697832.07
               2013-12-31   3128226.90
               2014-12-31   3884546.08
               2015-12-31   3511450.44
               2016-12-30   3574917.46
...                                ...
Western Europe 2019-12-31  11526084.85
               2020-12-31  12029185.51
               2021-12-31  11217097.30
               2022-12-30  11066267.48
               2023-12-29   9940536.04

[108 rows x 1 columns]

In [34]:
reg_prod_total.reset_index(inplace=True)
reg_prod_total

,ShipRegion,OrderDate,Total
0,British Isles,2012-12-31,1697832.07
1,British Isles,2013-12-31,3128226.90
2,British Isles,2014-12-31,3884546.08
3,British Isles,2015-12-31,3511450.44
4,British Isles,2016-12-30,3574917.46
...,...,...,...
103,Western Europe,2019-12-31,11526084.85
104,Western Europe,2020-12-31,12029185.51
105,Western Europe,2021-12-31,11217097.30
106,Western Europe,2022-12-30,11066267.48


In [35]:
reg_prod_total["Year"] = reg_prod_total["OrderDate"].dt.year
reg_prod_total

,ShipRegion,OrderDate,Total,Year
0,British Isles,2012-12-31,1697832.07,2012
1,British Isles,2013-12-31,3128226.90,2013
2,British Isles,2014-12-31,3884546.08,2014
3,British Isles,2015-12-31,3511450.44,2015
4,British Isles,2016-12-30,3574917.46,2016
...,...,...,...,...
103,Western Europe,2019-12-31,11526084.85,2019
104,Western Europe,2020-12-31,12029185.51,2020
105,Western Europe,2021-12-31,11217097.30,2021
106,Western Europe,2022-12-30,11066267.48,2022


In [37]:
import plotly.express as px

fig = px.line(reg_prod_total, x="Year", y='Total', color="ShipRegion")
fig.show()

In [113]:
consulta = """
SELECT strftime('%Y', OrderDate) AS Year, 
       strftime('%m', OrderDate) AS Month,
       ShipRegion AS Region, 
       ProductName AS Product, 
       SUM(Precio_total) As Total
FROM (
       SELECT datetime(o.OrderDate) as OrderDate, o.ShipRegion, od.Quantity*od.UnitPrice*(1-od.Discount) AS Precio_total, p.ProductName
       FROM Orders o
       INNER JOIN [Order Details] od ON od.OrderID == o.OrderID
       INNER JOIN Products p ON p.ProductID = od.ProductID
)
GROUP BY Year, Month, Region, Product
ORDER BY 5 DESC
"""

data_df = pd.read_sql(consulta, con=connection)
data_df

,Year,Month,Region,Product,Total
0,2014,12,Western Europe,Côte de Blaye,222130.5
1,2015,08,Western Europe,Côte de Blaye,220022.5
2,2016,12,Western Europe,Côte de Blaye,209746.0
3,2013,07,Western Europe,Côte de Blaye,209482.5
4,2018,08,Western Europe,Côte de Blaye,192882.0
...,...,...,...,...,...
86605,2018,09,Scandinavia,Guaraná Fantástica,4.5
86606,2018,10,Northern Europe,Guaraná Fantástica,4.5
86607,2021,05,Eastern Europe,Guaraná Fantástica,4.5
86608,2023,08,Scandinavia,Guaraná Fantástica,4.5


In [128]:
product_profit = data_df.groupby(["Product","Region"]).agg(meanTotal=("Total", "mean"), stdTotal=("Total","std")).sort_values("meanTotal", ascending=False)
product_profit

meanTotal      stdTotal
Product                 Region                                      
Côte de Blaye           Western Europe   120245.512500  35830.588553
                        North America     71226.975625  24537.409206
                        South America     70392.475000  26489.913360
Thüringer Rostbratwurst Western Europe    55366.795390  15763.042641
Mishi Kobe Niku         Western Europe    44114.744118  12931.501600
...                                                ...           ...
Geitost                 Central America     223.130769    119.268353
                        Northern Europe     180.480315    108.326253
                        Scandinavia         153.272727     97.388931
Guaraná Fantástica      Eastern Europe      148.616883    108.727747
Geitost                 Eastern Europe       94.202899     55.984656

[693 rows x 2 columns]

In [129]:
region = "Western Europe" # North America

region_profit = data_df[data_df["Region"] == region].groupby(["Product"]).agg(meanTotal=("Total", "mean"), stdTotal=("Total","std")).sort_values("meanTotal", ascending=False)
region_profit

,meanTotal,stdTotal
Product,,
Côte de Blaye,120245.512500,35830.588553
Thüringer Rostbratwurst,55366.795390,15763.042641
Mishi Kobe Niku,44114.744118,12931.501600
Sir Rodney's Marmalade,37618.663235,10658.487694
Carnarvon Tigers,28043.060662,8444.746568
...,...,...
Rhönbräu Klosterbier,3454.857684,978.098859
Filo Mix,3281.620588,945.980091
Konbu,2752.252941,775.857977


In [130]:
fig = px.bar(region_profit, y='meanTotal', error_y="stdTotal")
fig.show()

### **2. Análisis de Desempeño de Productos**
Analiza el desempeño de los productos de la empresa. Examina las ventas de productos por categoría, identificando los productos más vendidos y los menos vendidos. 

Pista: Utiliza gráficos de barras y de tarta para visualizar la participación de mercado de cada categoría y los ingresos generados por los productos individuales.

In [131]:
# !pip install 'ibis-framework[sqlite]'

In [4]:
import ibis

# Para ejecución interactiva
ibis.options.interactive = True

con = ibis.sqlite.connect('data/northwind.db')

In [7]:
categories = con.table("Categories")
products = con.table("Products")

joined_products = categories.join(products, "CategoryID")
joined_products

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ CategoryID ┃ CategoryName ┃ Description                                                ┃ Picture                                                                                                                                                                                                                                                               ┃ ProductID ┃ ProductName                     ┃ SupplierID ┃ QuantityPerUnit     ┃ UnitPrice ┃ UnitsInStock ┃ UnitsOnOrder ┃ ReorderLevel ┃ Discontinued ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ int64      │ string       │ string                                                     │ binary                                                                                                                                                                                                                                                                │ int64     │ string                          │ int64      │ string              │ decimal   │ int64        │ int64        │ int64        │ string       │
├────────────┼──────────────┼────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────┼─────────────────────────────────┼────────────┼─────────────────────┼───────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│          1 │ Beverages    │ Soft drinks, coffees, teas, beers, and ales                │ b"\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00P\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\nc\x00\x00\x10\xe8\x00\x00\x19\xf4\x00\x00'\xa4\xff"+10071  │         1 │ Chai                            │          1 │ 10 boxes x 20 bags  │     18.00 │           39 │            0 │           10 │ 0            │
│          1 │ Beverages    │ Soft drinks, coffees, teas, beers, and ales                │ b"\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00P\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\nc\x00\x00\x10\xe8\x00\x00\x19\xf4\x00\x00'\xa4\xff"+10071  │         2 │ Chang                           │          1 │ 24 - 12 oz bottles  │     19.00 │           17 │           40 │           25 │ 0            │
│          2 │ Condiments   │ Sweet and savory sauces, relishes, spreads, and seasonings │ b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00L\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\n\xda\x00\x00\x14\xef\x00\x00 ,\x00\x00/H\xff'+12027        │         3 │ Aniseed Syrup                   │          1 │ 12 - 550 ml bottles │     10.0

In [134]:
products.schema()

ibis.Schema {
  ProductID        !int64
  ProductName      !string
  SupplierID       int64
  CategoryID       int64
  QuantityPerUnit  string
  UnitPrice        decimal
  UnitsInStock     int64
  UnitsOnOrder     int64
  ReorderLevel     int64
  Discontinued     !string
}

In [135]:
products[["ProductID","ProductName"]].head(5)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ProductID ┃ ProductName                  ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !int64    │ !string                      │
├───────────┼──────────────────────────────┤
│         1 │ Chai                         │
│         2 │ Chang                        │
│         3 │ Aniseed Syrup                │
│         4 │ Chef Anton's Cajun Seasoning │
│         5 │ Chef Anton's Gumbo Mix       │
└───────────┴──────────────────────────────┘

In [9]:
ibis.options.interactive = False

products[["ProductID","ProductName"]].head(5)

r0 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

r1 := Project[r0]
  ProductID:   r0.ProductID
  ProductName: r0.ProductName

Limit[r1, n=5]

In [137]:
ibis.to_sql(products[["ProductID","ProductName"]].head(5))

```sql
SELECT
  "t0"."ProductID",
  "t0"."ProductName"
FROM "Products" AS "t0"
LIMIT 5
```

Pasamos a Pandas (en nuestra máquina)

In [138]:
category_df = joined_products.select(["CategoryName", "ProductName"]).to_pandas()
category_df.rename(columns={"ProductName": "Product"}, inplace=True)
category_df.set_index("Product", inplace=True)

In [139]:
product_profit.reset_index(inplace=True)
product_profit.set_index("Product", inplace=True)

In [140]:
product_profit

,Region,meanTotal,stdTotal
Product,,,
Côte de Blaye,Western Europe,120245.512500,35830.588553
Côte de Blaye,North America,71226.975625,24537.409206
Côte de Blaye,South America,70392.475000,26489.913360
Thüringer Rostbratwurst,Western Europe,55366.795390,15763.042641
Mishi Kobe Niku,Western Europe,44114.744118,12931.501600
...,...,...,...
Geitost,Central America,223.130769,119.268353
Geitost,Northern Europe,180.480315,108.326253
Geitost,Scandinavia,153.272727,97.388931


Le añadimos el dato de categoría

In [141]:
category_df.join(product_profit).sort_values("meanTotal", ascending=False).head(10)

,CategoryName,Region,meanTotal,stdTotal
Product,,,,
Côte de Blaye,Beverages,Western Europe,120245.512500,35830.588553
Côte de Blaye,Beverages,North America,71226.975625,24537.409206
Côte de Blaye,Beverages,South America,70392.475000,26489.913360
Thüringer Rostbratwurst,Meat/Poultry,Western Europe,55366.795390,15763.042641
Mishi Kobe Niku,Meat/Poultry,Western Europe,44114.744118,12931.501600
Côte de Blaye,Beverages,Southern Europe,37932.201493,16579.994957
Sir Rodney's Marmalade,Confections,Western Europe,37618.663235,10658.487694
Côte de Blaye,Beverages,British Isles,36764.149254,18923.969776
Thüringer Rostbratwurst,Meat/Poultry,South America,33102.532382,12150.048244


Fijaros en que no son objetos del mismo tipo

In [142]:
type(category_df)

pandas.core.frame.DataFrame

In [143]:
type(joined_products)

ibis.expr.types.joins.Join

In [144]:
ibis.to_sql(joined_products)

```sql
SELECT
  "t2"."CategoryID",
  "t2"."CategoryName",
  "t2"."Description",
  "t2"."Picture",
  "t3"."ProductID",
  "t3"."ProductName",
  "t3"."SupplierID",
  "t3"."QuantityPerUnit",
  "t3"."UnitPrice",
  "t3"."UnitsInStock",
  "t3"."UnitsOnOrder",
  "t3"."ReorderLevel",
  "t3"."Discontinued"
FROM "Categories" AS "t2"
INNER JOIN "Products" AS "t3"
  ON "t2"."CategoryID" = "t3"."CategoryID"
```

Vemos que algunos datos tienen problemas a la hora de mappear los datos a tipos válidos (mala creación de datos)

In [145]:
con.table("Orders")

KeyError: <Type.DATETIME: 'DATETIME'>

En este caso estaremos obligados a realizar una `select` explicita para extraer los datos como si fuera texto y procesarlo mediante parsers locales.

In [21]:
orders = con.sql("SELECT CAST(OrderDate as Text) as OrderDate, OrderID FROM Orders")
orders

SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text) as OrderDate, OrderID FROM Orders
  schema:
    OrderDate string
    OrderID   int64

In [22]:
df_orders = orders.to_pandas()
df_orders

,OrderDate,OrderID
0,2016-07-04,10248
1,2016-07-05,10249
2,2016-07-08,10250
3,2016-07-08,10251
4,2016-07-09,10252
...,...,...
16277,2012-12-26 04:58:22,26525
16278,2022-08-05 06:33:39,26526
16279,2022-02-09 08:20:12,26527
16280,2020-04-27 23:05:30,26528


In [26]:
pd.to_datetime(df_orders['OrderDate'],format='mixed')

0       2016-07-04 00:00:00
1       2016-07-05 00:00:00
2       2016-07-08 00:00:00
3       2016-07-08 00:00:00
4       2016-07-09 00:00:00
                ...        
16277   2012-12-26 04:58:22
16278   2022-08-05 06:33:39
16279   2022-02-09 08:20:12
16280   2020-04-27 23:05:30
16281   2023-01-10 05:40:51
Name: OrderDate, Length: 16282, dtype: datetime64[ns]

Ibis intenta encontrar el mejor encaje entre ambos tipos de datos, el de la base de datos origen y el del objeto pandas.

In [17]:
categories

DatabaseTable: Categories
  CategoryID   int64
  CategoryName string
  Description  string
  Picture      binary

In [18]:
df_categories = categories.to_pandas()
df_categories.dtypes

CategoryID       int64
CategoryName    object
Description     object
Picture         object
dtype: object

In [19]:
df_categories

,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
2,3,Confections,"Desserts, candies, and sweet breads",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
3,4,Dairy Products,Cheeses,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
5,6,Meat/Poultry,Prepared meats,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
6,7,Produce,Dried fruit and bean curd,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...
7,8,Seafood,Seaweed and fish,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...


In [12]:
# Si no está en modo interactivo, será necesario el `.to_pandas()`

orders.head(4).to_pandas()

,CAST(OrderDate as Text),OrderID
0,2016-07-04,10248
1,2016-07-05,10249
2,2016-07-08,10250
3,2016-07-08,10251


In [13]:
order_details = con.table("Order Details")
order_details

DatabaseTable: Order Details
  OrderID   !int64
  ProductID !int64
  UnitPrice !decimal
  Quantity  !int64
  Discount  !float64

In [14]:
p_o_details = orders.join(order_details, ["OrderID"], how="inner").join(products, ["ProductID"]).join(categories, ["CategoryID"])
ibis.to_sql(p_o_details)

```sql
SELECT
  "t4"."CAST(OrderDate as Text)",
  "t4"."OrderID",
  "t5"."ProductID",
  "t5"."UnitPrice",
  "t5"."Quantity",
  "t5"."Discount",
  "t6"."ProductName",
  "t6"."SupplierID",
  "t6"."CategoryID",
  "t6"."QuantityPerUnit",
  "t6"."UnitPrice" AS "UnitPrice_right",
  "t6"."UnitsInStock",
  "t6"."UnitsOnOrder",
  "t6"."ReorderLevel",
  "t6"."Discontinued",
  "t7"."CategoryName",
  "t7"."Description",
  "t7"."Picture"
FROM (
  SELECT
    CAST(OrderDate AS TEXT),
    OrderID
  FROM Orders
) AS "t4"
INNER JOIN "Order Details" AS "t5"
  ON "t4"."OrderID" = "t5"."OrderID"
INNER JOIN "Products" AS "t6"
  ON "t5"."ProductID" = "t6"."ProductID"
INNER JOIN "Categories" AS "t7"
  ON "t6"."CategoryID" = "t7"."CategoryID"
```

In [15]:
p_o_details_agg = p_o_details.group_by(["CategoryName","ProductName"]).aggregate(
    num_orders=p_o_details["OrderID"].count(), 
    total=(p_o_details["Quantity"]*p_o_details["UnitPrice"]*(1-p_o_details["Discount"])).sum())
p_o_details_agg = p_o_details_agg.order_by(p_o_details_agg["num_orders"].desc(), p_o_details_agg["total"].desc())
p_o_details_agg

r0 := SQLQueryResult
  query:
    SELECT CAST(OrderDate as Text), OrderID FROM Orders
  schema:
    CAST(OrderDate as Text) string
    OrderID                 int64

r1 := DatabaseTable: Order Details
  OrderID   !int64
  ProductID !int64
  UnitPrice !decimal
  Quantity  !int64
  Discount  !float64

r2 := DatabaseTable: Products
  ProductID       !int64
  ProductName     !string
  SupplierID      int64
  CategoryID      int64
  QuantityPerUnit string
  UnitPrice       decimal
  UnitsInStock    int64
  UnitsOnOrder    int64
  ReorderLevel    int64
  Discontinued    !string

r3 := DatabaseTable: Categories
  CategoryID   int64
  CategoryName string
  Description  string
  Picture      binary

r4 := JoinChain[r0]
  JoinLink[inner, r1]
    r0.OrderID == r1.OrderID
  JoinLink[inner, r2]
    r1.ProductID == r2.ProductID
  JoinLink[inner, r3]
    r2.CategoryID == r3.CategoryID
  values:
    CAST(OrderDate as Text): r0['CAST(OrderDate as Text)']
    OrderID:                 r0.OrderID
    ProductID:               r1.ProductID
    UnitPrice:               r1.UnitPrice
    Quantity:                r1.Quantity
    Discount:                r1.Discount
    ProductName:             r2.ProductName
    SupplierID:              r2.SupplierID
    CategoryID:              r2.CategoryID
    QuantityPerUnit:         r2.QuantityPerUnit
    UnitPrice_right:         r2.UnitPrice
    UnitsInStock:            r2.UnitsInStock
    UnitsOnOrder:            r2.UnitsOnOrder
    ReorderLevel:            r2.ReorderLevel
    Discontinued:            r2.Discontinued
    CategoryName:            r3.CategoryName
    Description:             r3.Description
    Picture:                 r3.Picture

r5 := Aggregate[r4]
  groups:
    CategoryName: r4.CategoryName
    ProductName:  r4.ProductName
  metrics:
    num_orders: Count(r4.OrderID)
    total:      Sum(r4.Quantity * r4.UnitPrice * 1 - r4.Discount)

Sort[r5]
  desc r5.num_orders
  desc r5.total

In [16]:
ibis.to_sql(p_o_details_agg)

```sql
SELECT
  *
FROM (
  SELECT
    "t8"."CategoryName",
    "t8"."ProductName",
    COUNT("t8"."OrderID") AS "num_orders",
    SUM((
      "t8"."Quantity" * "t8"."UnitPrice"
    ) * (
      1 - "t8"."Discount"
    )) AS "total"
  FROM (
    SELECT
      "t4"."CAST(OrderDate as Text)",
      "t4"."OrderID",
      "t5"."ProductID",
      "t5"."UnitPrice",
      "t5"."Quantity",
      "t5"."Discount",
      "t6"."ProductName",
      "t6"."SupplierID",
      "t6"."CategoryID",
      "t6"."QuantityPerUnit",
      "t6"."UnitPrice" AS "UnitPrice_right",
      "t6"."UnitsInStock",
      "t6"."UnitsOnOrder",
      "t6"."ReorderLevel",
      "t6"."Discontinued",
      "t7"."CategoryName",
      "t7"."Description",
      "t7"."Picture"
    FROM (
      SELECT
        CAST(OrderDate AS TEXT),
        OrderID
      FROM Orders
    ) AS "t4"
    INNER JOIN "Order Details" AS "t5"
      ON "t4"."OrderID" = "t5"."OrderID"
    INNER JOIN "Products" AS "t6"
      ON "t5"."ProductID" = "t6"."ProductID"
    INNER JOIN "Categories" AS "t7"
      ON "t6"."CategoryID" = "t7"."CategoryID"
  ) AS "t8"
  GROUP BY
    1,
    2
) AS "t9"
ORDER BY
  "t9"."num_orders" DESC,
  "t9"."total" DESC
```

In [151]:
p_o_details_agg.to_pandas()

,CategoryName,ProductName,num_orders,total
0,Condiments,Louisiana Hot Spiced Okra,8040,3504941
1,Confections,Teatime Chocolate Biscuits,8024,1889691.4200000037
2,Beverages,Outback Lager,8020,3064462.65
3,Confections,Sir Rodney's Marmalade,7999,16653807.36
4,Confections,Gumbär Gummibärchen,7999,6391018.984499998
...,...,...,...,...
72,Produce,Tofu,7834,4644738.99
73,Confections,Scottish Longbreads,7818,2528089
74,Meat/Poultry,Perth Pasties,7817,6561681.369999989
75,Dairy Products,Mascarpone Fabioli,7817,6381012.16


In [153]:
consulta= p_o_details_agg.group_by("CategoryName").aggregate(category_total=p_o_details_agg["total"].sum()).order_by(ibis.desc("category_total"))
consulta.to_pandas()

,CategoryName,category_total
0,Beverages,92163184.17999999
1,Confections,66337803.06499998
2,Meat/Poultry,64881147.969499916
3,Dairy Products,58018116.78500001
4,Condiments,55795126.78499997
5,Seafood,49921604.16749999
6,Produce,32701119.88000001
7,Grains/Cereals,28568530.3375


In [154]:
ibis.to_sql(consulta)

```sql
SELECT
  *
FROM (
  SELECT
    "t10"."CategoryName",
    SUM("t10"."total") AS "category_total"
  FROM (
    SELECT
      *
    FROM (
      SELECT
        "t8"."CategoryName",
        "t8"."ProductName",
        COUNT("t8"."OrderID") AS "num_orders",
        SUM((
          "t8"."Quantity" * "t8"."UnitPrice"
        ) * (
          1 - "t8"."Discount"
        )) AS "total"
      FROM (
        SELECT
          "t4"."CAST(OrderDate as Text)",
          "t4"."OrderID",
          "t5"."ProductID",
          "t5"."UnitPrice",
          "t5"."Quantity",
          "t5"."Discount",
          "t6"."ProductName",
          "t6"."SupplierID",
          "t6"."CategoryID",
          "t6"."QuantityPerUnit",
          "t6"."UnitPrice" AS "UnitPrice_right",
          "t6"."UnitsInStock",
          "t6"."UnitsOnOrder",
          "t6"."ReorderLevel",
          "t6"."Discontinued",
          "t7"."CategoryName",
          "t7"."Description",
          "t7"."Picture"
        FROM (
          SELECT
            CAST(OrderDate AS TEXT),
            OrderID
          FROM Orders
        ) AS "t4"
        INNER JOIN "Order Details" AS "t5"
          ON "t4"."OrderID" = "t5"."OrderID"
        INNER JOIN "Products" AS "t6"
          ON "t5"."ProductID" = "t6"."ProductID"
        INNER JOIN "Categories" AS "t7"
          ON "t6"."CategoryID" = "t7"."CategoryID"
      ) AS "t8"
      GROUP BY
        1,
        2
    ) AS "t9"
    ORDER BY
      "t9"."num_orders" DESC,
      "t9"."total" DESC
  ) AS "t10"
  GROUP BY
    1
) AS "t11"
ORDER BY
  "t11"."category_total" DESC
```

Una vez sabemos cómo obtener el dato, solo debemos representarlo gráficamente.

In [155]:
precio_por_categoría = consulta.to_pandas()

fig = px.bar(precio_por_categoría, x='CategoryName', y="category_total")
fig.show()

In [157]:
precio_producto= p_o_details_agg.to_pandas()

fig = px.bar(precio_producto, x='ProductName', y="total", color="CategoryName")
fig.show()

### **3. Análisis de Rentabilidad de Clientes**
Analiza la rentabilidad de los clientes de la empresa. Examina los ingresos generados por cada cliente, así como los costos asociados con el servicio o producto proporcionado. 

Pista: Utiliza gráficos de dispersión y tablas dinámicas para identificar los clientes más rentables y aquellos que requieren mayor atención.

In [158]:
summary_per_quarter = con.sql("SELECT CAST(ShippedDate as Text) as ShippedDate, OrderID, Subtotal FROM [Summary of Sales by Quarter]")
orders = con.sql("SELECT CustomerID, OrderID FROM Orders")
customers = con.table("customers")

joined_data = summary_per_quarter.join(orders, "OrderID").join(customers, "CustomerID")
joined_data.schema()

ibis.Schema {
  ShippedDate   string
  OrderID       int64
  Subtotal      float64
  CustomerID    string
  CompanyName   string
  ContactName   string
  ContactTitle  string
  Address       string
  City          string
  Region        string
  PostalCode    string
  Country       string
  Phone         string
  Fax           string
}

In [159]:
per_customer = joined_data.aggregate(
        by="CompanyName", 
        total=joined_data["Subtotal"].sum(), 
        dev=joined_data["Subtotal"].std()
    ).order_by(ibis.desc("total")).to_pandas()

per_customer

,CompanyName,total,dev
0,IT,9.745371e+06,17911.792434
1,B's Beverages,6.154115e+06,17822.000552
2,Hungry Coyote Import Store,5.698024e+06,17304.451978
3,Rancho grande,5.559034e+06,17799.147451
4,Gourmet Lanchonetes,5.552310e+06,17403.553505
...,...,...,...
87,Reggiani Caseifici,4.222835e+06,18400.755160
88,Lehmanns Marktstand,4.182514e+06,18467.964511
89,Furia Bacalhau e Frutos do Mar,4.098648e+06,17470.622273
90,Océano Atlántico Ltda.,4.059079e+06,17213.546773


In [160]:
fig = px.bar(per_customer, x='CompanyName', y="total", error_y="dev")
fig.show()

In [161]:
IT_condition = joined_data.CompanyName == "IT"

ibis.to_sql(joined_data[IT_condition])

```sql
SELECT
  *
FROM (
  SELECT
    "t3"."ShippedDate",
    "t3"."OrderID",
    "t3"."Subtotal",
    "t4"."CustomerID",
    "t5"."CompanyName",
    "t5"."ContactName",
    "t5"."ContactTitle",
    "t5"."Address",
    "t5"."City",
    "t5"."Region",
    "t5"."PostalCode",
    "t5"."Country",
    "t5"."Phone",
    "t5"."Fax"
  FROM (
    SELECT
      CAST(ShippedDate AS TEXT) AS ShippedDate,
      OrderID,
      Subtotal
    FROM "Summary of Sales by Quarter"
  ) AS "t3"
  INNER JOIN (
    SELECT
      CustomerID,
      OrderID
    FROM Orders
  ) AS "t4"
    ON "t3"."OrderID" = "t4"."OrderID"
  INNER JOIN "customers" AS "t5"
    ON "t4"."CustomerID" = "t5"."CustomerID"
) AS "t6"
WHERE
  "t6"."CompanyName" = 'IT'
```

In [162]:
df_IT_data = joined_data[IT_condition].to_pandas()
df_IT_data["ShippedDate"] = pd.to_datetime(df_IT_data["ShippedDate"])

extract = df_IT_data[["ShippedDate","Subtotal"]].set_index("ShippedDate")

extract_subsampled = extract.resample("BYE").mean("Subtotal")

In [163]:
fig = px.area(extract_subsampled, x=extract_subsampled.index, y="Subtotal")
fig.show()

### **4. Análisis de Inventarios y Proveedores**
Analiza los niveles de inventario y la gestión de proveedores de la empresa. Examina el inventario disponible por categoría de productos, así como el rendimiento de los proveedores en términos de puntualidad de entrega y calidad de los productos. 

Pista: Utiliza gráficos de líneas y tablas dinámicas para mostrar tendencias y comparar el desempeño de los proveedores.


### **5. Análisis de Comportamiento de Empleados**
Analiza el comportamiento de los empleados de la empresa. Examina las ventas realizadas por cada empleado, así como su eficiencia en el manejo de pedidos y su puntualidad en la entrega de productos. 

Pista: Utiliza gráficos de barra y de dispersión para comparar el desempeño de los empleados y identificar áreas de mejora.

**NOTA** Para poder realizar estas operaciones de forma correcta deberemos alterar la vista invoices, Salesperson será únicamente el nombre del empleado.

In [46]:
ventas = con.sql("SELECT Salesperson, ExtendedPrice FROM Invoices")
ventas

SQLQueryResult
  query:
    SELECT Salesperson, ExtendedPrice FROM Invoices
  schema:
    Salesperson   string
    ExtendedPrice float64

In [48]:
empleados_rentables = ventas.aggregate(by="Salesperson", rendimiento=ventas["ExtendedPrice"].sum()).order_by(ibis.desc("rendimiento"))

In [50]:
ibis.to_sql(empleados_rentables)

```sql
SELECT
  *
FROM (
  SELECT
    "t0"."Salesperson",
    SUM("t0"."ExtendedPrice") AS "rendimiento"
  FROM (
    SELECT
      Salesperson,
      ExtendedPrice
    FROM Invoices
  ) AS "t0"
  GROUP BY
    1
) AS "t1"
ORDER BY
  "t1"."rendimiento" DESC
```

In [51]:
empleados_rentables.to_pandas()

,Salesperson,rendimiento
0,Margaret,5.148840e+07
1,Steven,5.138646e+07
2,Janet,5.044557e+07
3,Nancy,4.965942e+07
4,Robert,4.965190e+07
5,Laura,4.928114e+07
6,Michael,4.913997e+07
7,Anne,4.901968e+07
8,Andrew,4.831410e+07
